In [1]:
#!pip install pydantic[email]

In [2]:
import syft as sy
import numpy as np
from syft.core.adp.entity import DataSubject

Seeding DB with roles!


In [3]:
new_domain = sy.login(email="info@openmined.org", password="changethis", port=8081)

Connecting to http://localhost:8081... done! 	 Logging into adp... done!


In [4]:
ishan = DataSubject(name="Ishan")

In [5]:
data = sy.Tensor(np.random.rand(1, 10)).private(0, 10, entity=ishan)
data

Tensor(child=SingleEntityPhiTensor(entity=Ishan, child=[[0.2016122  0.63649143 0.3963239  0.27374393 0.87569666 0.04004367
  0.17204079 0.20989991 0.75958011 0.44298358]]))

In [6]:
data.child.init_pointer(client=new_domain)

<SingleEntityPhiTensorPointer -> adp:1af10f93549245d78fe65dda23b1fd8c>

In [7]:
se_ptr = data.child.init_pointer(client=new_domain)

In [8]:
type(se_ptr)

syft.proxy.syft.core.tensor.tensor.SingleEntityPhiTensorPointer

In [9]:
sy.serialize(se_ptr)

child {
  arrow_data: "\377\377\377\377\370\000\000\000\024\000\000\000\000\000\000\000\014\000\032\000\006\000\005\000\010\000\014\000\014\000\000\000\000\004\004\000 \000\000\000P\000\000\000\000\000\000\000\000\000\000\000\000\000\016\000(\000\007\000\010\000\014\000\020\000\024\000\016\000\000\000\000\000\000\003\214\000\000\0004\000\000\000\030\000\000\000\000\000\000\000\000\000\000\000P\000\000\000\000\000\000\000\000\000\000\000\002\000\000\000P\000\000\000\000\000\000\000\010\000\000\000\000\000\000\000\000\000\000\000\002\000\000\0000\000\000\000\014\000\000\000\010\000\020\000\010\000\004\000\010\000\000\000\014\000\000\000\n\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\010\000\024\000\010\000\004\000\010\000\000\000\020\000\000\000\001\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\006\000\010\000\006\000\006\000\000\000\000\000\002\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000

In [10]:
serialized_ptr = sy.serialize(se_ptr)

In [11]:
type(serialized_ptr)

proto.core.tensor.single_entity_phi_tensor_pb2.SingleEntityPhiTensorPointer

In [12]:
# Debugging deserialization issues --> Notice that all attributes can deserialize but the object itself can't. Hm

proto = serialized_ptr
[sy.deserialize(proto.entity, from_proto=True),
sy.deserialize(proto.child),
sy.deserialize(proto.min_vals),
sy.deserialize(proto.max_vals),
sy.deserialize(proto.location)
,sy.deserialize(proto.scalar_manager, from_bytes=True)
,sy.deserialize(proto.id_at_location, from_proto=True)
,sy.deserialize(proto.object_type, from_bytes=True)
,sy.deserialize(proto.tags, from_bytes=True)
]

[<DataSubject:Ishan>,
 array([[0.2016122 , 0.63649143, 0.3963239 , 0.27374393, 0.87569666,
         0.04004367, 0.17204079, 0.20989991, 0.75958011, 0.44298358]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[10., 10., 10., 10., 10., 10., 10., 10., 10., 10.]]),
 <Address - Domain:<SpecificLocation:..0bb86>>,
 <UID: a2cbcb2fd5ca45e4acd39c29780a5574>,
 '',
 '']

In [18]:
# Verifying that we can reconstruct the SEPT Pointer with the deserialized attributes
proto = serialized_ptr
entity = sy.deserialize(proto.entity, from_proto=True)
child = sy.deserialize(proto.child)
min_vals = sy.deserialize(proto.min_vals)
max_vals = sy.deserialize(proto.max_vals)
client = sy.deserialize(proto.location)
scalar_manager = sy.deserialize(proto.scalar_manager, from_bytes=True)
id_at_location = sy.deserialize(proto.id_at_location, from_proto=True)
object_type = sy.deserialize(proto.object_type, from_bytes=True)
tags = sy.deserialize(proto.tags, from_bytes=True)

sy.core.tensor.autodp.single_entity_phi.SingleEntityPhiTensorPointer(
            child=child,
            entity=entity,
            min_vals=min_vals,
            max_vals=max_vals,
            client=client,
            scalar_manager=scalar_manager,
            id_at_location=id_at_location,
            object_type=object_type,
            tags=tags,
        )

<SingleEntityPhiTensorPointer -> adp:a2cbcb2fd5ca45e4acd39c29780a5574>

In [14]:
sy.deserialize(serialized_ptr)

[2021-08-26T18:52:47.214181-0400][CRITICAL][logger]][17565] You tried to deserialize an unsupported type. This can be caused by several reasons. Either you are actively writing Syft code and forgot to create one, or you are trying to deserialize an object which was serialized using a different version of Syft and the object you tried to deserialize is not supported in this version.


TypeError: You tried to deserialize an unsupported type. This can be caused by several reasons. Either you are actively writing Syft code and forgot to create one, or you are trying to deserialize an object which was serialized using a different version of Syft and the object you tried to deserialize is not supported in this version.